In [2]:
import pandas as pd
import json
import requests
import numpy as np

### Write the pageviews of every actor

In [3]:
# import dataset
# Character metadata
char_md_cols = ['Wikipedia_movie_ID',
'Freebase_movie_ID',
'Movie_release_date',
'Character_name',
'Actor_date_of_birth',
'Actor_gender',
'Actor_height_(in_meters)',
'Actor_ethnicity_(Freebase_ID)',
'Actor_name',
'Actor_age_at_movie_release',
'Freebase_character/actor_map_ID',
'Freebase_character_ID',
'Freebase_actor_ID',
]
char_md = pd.read_csv('data/character.metadata.tsv', sep='\t', names=char_md_cols)
print('Number of characters:', char_md.shape[0])
char_md.head(2)

Number of characters: 450669


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height_(in_meters),Actor_ethnicity_(Freebase_ID),Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


In [16]:
def get_pageviews(actor_name: str, first_timestamp='20170101', second_timestamp='20200101', granularity="monthly"):
    actor_name = actor_name.replace(' ', '_')

    url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/" + \
        actor_name + "/" + granularity + "/" + first_timestamp + "/" + second_timestamp
    
    response = requests.get(url, headers={'User-Agent': ''})
    if response.status_code != 200:
        return np.nan
    
    views = response.json()['items']
    tot_views = sum(item['views'] for item in views)
    return tot_views


In [13]:
male_char = char_md[char_md["Actor_gender"] == "M"]
female_char = char_md[char_md["Actor_gender"] == "F"]

male_actor_list = list(male_char.groupby("Actor_name").indices.keys())
female_actor_list = list(female_char.groupby("Actor_name").indices.keys())

In [17]:
# requests for male actors
male_actor_pageviews = {}

for actor in male_actor_list:
    male_actor_pageviews[actor] = get_pageviews(actor)

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [ ]:
# requests for male actors
male_actor_pageviews = {}

for actor in male_actor_list:
    male_actor_pageviews[actor] = get_pageviews(actor)

In [27]:
# requests for male actors
female_actor_pageviews = {}

i = 0
for actor in female_actor_list:
    i = i + 1
    if i % 1000 == 0:
        print(i)
    female_actor_pageviews[actor] = get_pageviews(actor)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


In [24]:
# convert dictionaries to files (male actors)

with open('male_actor_pageviews.csv', 'w', encoding='utf-8') as f:
    f.write('Actor_name,Pageviews\n')
    for key, value in male_actor_pageviews.items():
        if value == np.nan:
            continue
        string = key + "," + str(value) + "\n"
        f.write(string)

In [31]:
# convert dictionaries to files (female actors)

with open('female_actor_pageviews.csv', 'w', encoding='utf-8') as f:
    f.write('Actor_name,Pageviews\n')
    for key, value in female_actor_pageviews.items():
        string = key + "," + str(value) + "\n"
        f.write(string)

In [18]:
len(male_actor_pageviews)

40925